In [ ]:
# Experiment: This cell will attempt to read the file using pandas - I like the output! 
pip install pandas
import pandas
df = pandas.read_csv('InstantChurchDirectory-20210218-042000.csv')
# print(df)
print(df[['First_Name', 'Phone_Numbers', 'Emails']])

In [ ]:
# Function: parse_contact_info: This cell will create a function to strip info for each person
#

def parse_contact_info(name, contact_medium, contact_list):
    # This function will return either the phone number or an email address for a person    
    filter_object = filter(lambda a: name in a, contact_list)
    contact_hit = list(filter_object)
    str1 = ""
    str2 = str1.join(contact_hit)
    contact_info = str2.split(" ")
    mydict = {'owner': name, contact_medium: contact_info[0]}
    
    return mydict

# mail1 = ['CalebAnderson@gmail.com (Caleb)', 'KateSLAnderson@gmail.com (Katy)']
# phone1 = ['515-230-2171 (Caleb), 615-310-7347 (Katy)']

# person_email_contact = parse_contact_info('Tom', 'email', mail1)
# person_phone_contact = parse_contact_info('Tom', 'phone', phone1)
# person_contact_info = person_email_contact | person_phone_contact
# print(person_contact_info)


In [ ]:
# Function: join_names: This cell will create a function to combine first_name and child_names into a list of names
#
import re
def join_names(first_name, child_name):
    # This function will combine all of the names into a list   
    str1 = ''
    str2 = str1.join(first_name)
    str3 = re.findall(r'(\S+)', str2)
    # print(len(str3))
    if len(str3)==3:
        del str3[1]
    str4 = str1.join(child_name)
    str5 = re.findall(r'(\S+)',str4)
    return str3 + str5

# first_n = ['Caleb and Katy']
# child_n = ['Aili, Hazel, Ettalyn']
# result = join_names(first_n, child_n)
# print(result)


In [16]:
# This cell will attmept to parse out the child names using csv
# https://stackoverflow.com/questions/12974866/split-delimited-string-cell-value-into-multiple-rows
# Last_Name,First_Name,Child_Names,Address,City,State,ZIP,Phone_Numbers,Emails,AnniversaryDate,Additional_Details,Notes,FamilyId,Active,ImageExists,DateModified 
import csv

with open('InstantChurchDirectory-20210218-042000.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    new_row = {}
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            fieldnames = [key for key in row]
            
            print(fieldnames)
            print(type(fieldnames))            
            fieldnames.append('owner')
            fieldnames.append('email')
            fieldnames.append('phone')
            fieldnames.append('rest_of_it')
            print(fieldnames)
            print(type(fieldnames))

            line_count += 1
        # elif line_count <= 1:
        else:
            name_list = join_names(row['First_Name'], row['Child_Names'])
            
            
            mail_list = [x.strip() for x in row['Emails'].split(',')]
            phone_list = [x.strip() for x in row['Phone_Numbers'].split(',')]
            for each_name in name_list: 
                emaildict = parse_contact_info(each_name, 'email', mail_list)
                phonedict = parse_contact_info(each_name, 'phone', phone_list)
                new_row = emaildict | phonedict
                if (new_row['phone'] or new_row['email']):
                    print(new_row)
                    print(type(new_row))
                    new_row['rest_of_it'] = row
                    print(new_row.keys())
                
            # print(f'\t{child_list} {mail_list} {phone_list}')
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are Last_Name, First_Name, Child_Names, Address, City, State, ZIP, Phone_Numbers, Emails, AnniversaryDate, Additional_Details, Notes, FamilyId, Active, ImageExists, DateModified
['Last_Name', 'First_Name', 'Child_Names', 'Address', 'City', 'State', 'ZIP', 'Phone_Numbers', 'Emails', 'AnniversaryDate', 'Additional_Details', 'Notes', 'FamilyId', 'Active', 'ImageExists', 'DateModified']
<class 'list'>
['Last_Name', 'First_Name', 'Child_Names', 'Address', 'City', 'State', 'ZIP', 'Phone_Numbers', 'Emails', 'AnniversaryDate', 'Additional_Details', 'Notes', 'FamilyId', 'Active', 'ImageExists', 'DateModified', 'owner', 'email', 'phone', 'rest_of_it']
<class 'list'>
{'owner': 'Caleb', 'email': 'CalebAnderson@gmail.com', 'phone': '515-230-2171'}
<class 'dict'>
dict_keys(['owner', 'email', 'phone', 'rest_of_it'])
{'owner': 'Katy', 'email': 'KateSLAnderson@gmail.com', 'phone': '615-310-7347'}
<class 'dict'>
dict_keys(['owner', 'email', 'phone', 'rest_of_it'])
{'owner': 'Ethan', 'email'

In [17]:
# 
# This cell will attempt to use csv to write out the output
#
import csv

with open('employee_file2.csv', mode='w') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for row in new_row:
        writer.writerow(new_row)
    